# AI Market Trend Predictor (NIFTY 50)

**Project Track:** Market Trend Analysis / Financial Trend Prediction  
**Goal:** Predict the **next trading day market trend regime** as:
- 📈 Uptrend
- ➡️ Sideways
- 📉 Downtrend

This notebook demonstrates the end-to-end AI workflow:
1) Data download (yfinance)  
2) Feature engineering  
3) Trend labeling logic  
4) ML model inference (Random Forest)  
5) Evaluation and sample outputs  
6) Ethical considerations & responsible AI  
7) Conclusion & future scope


## 1. Problem Definition & Objective

### Selected Project Track
**AI in Market Trend Analysis** (Financial Analytics)

### Problem Statement
Financial markets are noisy and hard to interpret manually.  
The objective is to build an AI-based system that can classify the market into:
- **Uptrend**
- **Downtrend**
- **Sideways**

for the **next trading day**, using historical market data.

### Real-world Relevance & Motivation
Trend regimes are useful for:
- risk management
- portfolio allocation
- decision support for traders and analysts

This project is an **educational prototype** and does **not** provide buy/sell advice.


In [ ]:
# If running on Colab:
# !pip -q install yfinance pandas numpy scikit-learn matplotlib


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

import pickle
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


## 2. Data Understanding & Preparation

### Dataset Source
- Data is downloaded from **Yahoo Finance** using the **yfinance API**
- Example ticker: `^NSEI` (NIFTY 50 Index)

### Data Type
Daily OHLCV data:
- Open, High, Low, Close, Volume

### Data Preparation Steps
- Handle missing values
- Compute rolling indicators (moving averages, volatility)
- Generate model features for prediction


In [ ]:
ticker = "^NSEI"
start = "2015-01-01"
end = "2024-01-01"

raw = yf.download(ticker, start=start, end=end)

# Fix yfinance returning MultiIndex columns sometimes
if isinstance(raw.columns, pd.MultiIndex):
    raw.columns = raw.columns.get_level_values(0)

raw.head(), raw.shape


In [ ]:
print("Shape:", raw.shape)
raw.info()

plt.figure(figsize=(12,4))
plt.plot(raw.index, raw["Close"])
plt.title("NIFTY 50 Close Price")
plt.xlabel("Date")
plt.ylabel("Close")
plt.grid(True)
plt.show()


### Handling Missing Values / Noise
Rolling technical indicators require a minimum number of days.  
This creates missing values (NaNs), which are removed after computing indicators.

We also replace invalid values (`inf`, `-inf`) produced by calculations.


In [ ]:
def make_features(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()

    df["Return"] = df["Close"].pct_change()
    df["LogReturn"] = np.log(df["Close"] / df["Close"].shift(1))

    df["MA_20"] = df["Close"].rolling(20).mean()
    df["MA_50"] = df["Close"].rolling(50).mean()

    df["Volatility_20"] = df["Return"].rolling(20).std()
    df["Volatility_50"] = df["Return"].rolling(50).std()

    df["Momentum_10"] = df["Close"] - df["Close"].shift(10)
    df["ROC_10"] = df["Close"].pct_change(10)

    df["Vol_Change"] = df["Volume"].pct_change()
    df["Vol_MA_20"] = df["Volume"].rolling(20).mean()

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

feat = make_features(raw)
feat.head(), feat.shape


## 3. Model / System Design

### AI Technique Used
✅ **Supervised Machine Learning Classification**

### Pipeline
1. Download OHLCV data via API (yfinance)
2. Compute features (returns, volatility, moving averages, momentum, volume)
3. Generate **trend labels** using MA20 vs MA50 logic
4. Train an ML classifier (Random Forest) on labeled data
5. Predict trend regime for next trading day

### Justification of Design Choices
- **Random Forest** works well for tabular feature sets and noisy data
- Provides probability confidence via `predict_proba`
- Moving average based trend labeling is a standard finance heuristic


In [ ]:
TREND_MAP = {1: "Uptrend", 0: "Sideways", -1: "Downtrend"}

def create_trend_labels(df_feat: pd.DataFrame, threshold: float = 0.002) -> pd.DataFrame:
    df = df_feat.copy()
    diff = (df["MA_20"] - df["MA_50"]) / df["MA_50"]

    df["Trend"] = np.where(
        diff > threshold, 1,
        np.where(diff < -threshold, -1, 0)
    )
    df["Trend_Label"] = df["Trend"].map(TREND_MAP)
    return df

threshold = 0.002
labeled = create_trend_labels(feat, threshold=threshold)
labeled["Trend_Label"].value_counts()


In [ ]:
plt.figure(figsize=(12,4))
plt.plot(labeled.index, labeled["Close"], alpha=0.5, label="Close")

plt.scatter(
    labeled.index,
    labeled["Close"],
    c=labeled["Trend"],
    cmap="coolwarm",
    s=8
)

plt.title("Rule-Based Trend Label Overlay")
plt.xlabel("Date")
plt.ylabel("Close")
plt.grid(True)
plt.show()


## 4. Core Implementation

This notebook demonstrates the **prediction pipeline** using a trained model.

### Prediction target
The model is trained to predict the **next trading day trend** using today's feature vector.

### Requirement
The code must run **top-to-bottom without errors**.


In [ ]:
FEATURES = [
    "Return", "LogReturn",
    "Volatility_20", "Volatility_50",
    "Momentum_10", "ROC_10",
    "Vol_Change", "Vol_MA_20",
    "Volume"
]


In [ ]:
MODEL_PATH = "model/rf_nextday.pkl"

with open(MODEL_PATH, "rb") as f:
    model = pickle.load(f)

type(model)


In [ ]:
df_ml = labeled.copy()

# Predict next day trend (shift label)
df_ml["Target_NextDay"] = df_ml["Trend"].shift(-1)
df_ml.dropna(inplace=True)

X = df_ml[FEATURES]
y = df_ml["Target_NextDay"].astype(int)

X.shape, y.value_counts()


In [ ]:
split_idx = int(len(df_ml) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# If model already trained, evaluate directly:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["Downtrend", "Sideways", "Uptrend"]))

cm = confusion_matrix(y_test, y_pred, labels=[-1,0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Down", "Sideways", "Up"])
disp.plot()
plt.show()


In [ ]:
X_last = df_ml[FEATURES].iloc[[-1]]
pred_class = int(model.predict(X_last)[0])

confidence = None
if hasattr(model, "predict_proba"):
    proba = model.predict_proba(X_last)[0]
    class_to_prob = dict(zip(model.classes_, proba))
    confidence = float(class_to_prob.get(pred_class, 0.0))

output = {
    "ticker": ticker,
    "last_available_date": str(df_ml.index[-1].date()),
    "predicted_for": "next_trading_day",
    "trend_class": pred_class,
    "trend_label": TREND_MAP[pred_class],
    "confidence": confidence
}

output


## 5. Evaluation & Analysis

### Metrics used
- Precision
- Recall
- F1-score
- Confusion matrix

### Sample Outputs
The system outputs:
- predicted trend class (-1/0/1)
- predicted trend label
- confidence score (probability)

### Performance notes
Trend classification is challenging because:
- markets are noisy
- regimes change frequently
- sideways class is often dominant


## 6. Ethical Considerations & Responsible AI

### Bias and Fairness
- Market datasets can be biased by time period (bull vs bear markets)
- Sideways periods may dominate → class imbalance

### Dataset limitations
- Data pulled from yfinance may have:
  - missing values
  - adjusted data inconsistencies
  - dependency on API availability

### Responsible AI Use
- This model is NOT financial advice.
- It is meant as a decision-support / educational tool.
- Users should rely on professional judgement and risk controls.


## 7. Conclusion & Future Scope

### Conclusion
- Built an end-to-end AI pipeline for trend regime prediction.
- Developed a usable tool (backend + dashboard) using ML + visual analytics.

### Future Improvements
- Add multi-ticker watchlist predictions
- Add news sentiment / macro indicators
- Backtesting framework for strategy validation
- Explore deep learning time-series models (LSTM/Transformers)
- Deploy as cloud-hosted web application
